# Exercise 03 : Aggregations

## Import

In [36]:
import pandas as pd
import sqlite3

## Create a connection to the database using the library *sqlite3*

In [37]:
db_connect = sqlite3.connect('../data/checking-logs.sqlite.sqlite copy')

## Get the schema of the table test

In [38]:
query = "PRAGMA table_info(test);"
pd.read_sql(sql=query, con=db_connect)

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,uid,TEXT,0,None,0
2,2,labname,TEXT,0,None,0
3,3,first_commit_ts,TIMESTAMP,0,None,0
4,4,first_view_ts,TIMESTAMP,0,None,0


## Get only the first 10 rows of the table test

In [39]:
query = "SELECT * FROM test LIMIT 10;"
pd.read_sql(sql=query, con=db_connect, index_col='index')

,uid,labname,first_commit_ts,first_view_ts
index,,,,
3,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
4,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
7,user_30,laba04s,2020-04-18 14:51:37.498399,2020-04-17 22:46:26.785035
8,user_14,laba04,2020-04-18 15:14:00.312338,2020-04-18 10:53:52.623447
11,user_14,laba04s,2020-04-18 22:30:30.247628,2020-04-18 10:53:52.623447
18,user_19,laba04,2020-04-20 19:05:01.297780,2020-04-21 20:30:38.034966
19,user_25,laba04,2020-04-20 19:16:50.673054,2020-05-09 23:54:54.260791
20,user_21,laba04,2020-04-21 17:48:00.487806,2020-04-22 22:40:36.824081
21,user_30,project1,2020-04-22 12:36:24.053518,2020-04-17 22:46:26.785035


## Find among all the users the minimum value of the delta between the first commit of the user and the deadline of the corresponding lab

In [40]:
query = "PRAGMA table_info(deadlines);"
pd.read_sql(sql=query, con=db_connect)

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,labs,TEXT,0,None,0
2,2,deadlines,INTEGER,0,None,0


In [41]:
query = "SELECT * FROM deadlines LIMIT 10;"
pd.read_sql(sql=query, con=db_connect, index_col='index')

,labs,deadlines
index,,
0,laba04,1587945599
1,laba04s,1587945599
2,laba05,1588550399
4,laba06,1590364799
5,laba06s,1590364799
3,project1,1589673599


In [42]:
query="""
SELECT
    uid,
    MIN(diff)
FROM (
    SELECT
        test.uid,
        test.labname,
        CAST((julianday(test.first_commit_ts) - julianday(deadlines.deadlines, 'unixepoch')) * 24 AS INT) AS diff
    FROM
        test
    INNER JOIN
        deadlines
    ON
        test.labname != 'project1'
        AND test.labname = deadlines.labs
);
"""

df_min = pd.read_sql(sql=query, con=db_connect)
df_min

,uid,MIN(diff)
0,user_30,-202


## Find among all the users the maximum value of the delta between the first commit of the user and the deadline of the corresponding lab

In [43]:
query="""
SELECT
    uid,
    MAX(diff)
FROM (
    SELECT
        test.uid,
        test.labname,
        CAST((julianday(test.first_commit_ts) - julianday(deadlines.deadlines, 'unixepoch')) * 24 AS INT) AS diff
    FROM
        test
    INNER JOIN
        deadlines
    ON
        test.labname != 'project1'
        AND test.labname = deadlines.labs
);
"""

df_max = pd.read_sql(sql=query, con=db_connect)
df_max

,uid,MAX(diff)
0,user_25,-2


## Find among all the users the average value of the delta between the first commit of the user and the deadline of the corresponding lab

In [44]:
query="""
SELECT
    AVG(diff)
FROM (
    SELECT
        test.uid,
        test.labname,
        CAST((julianday(test.first_commit_ts) - julianday(deadlines.deadlines, 'unixepoch')) * 24 AS INT) AS diff
    FROM
        test
    INNER JOIN
        deadlines
    ON
        test.labname != 'project1'
        AND test.labname = deadlines.labs
);
"""

df_avg = pd.read_sql(sql=query, con=db_connect)
df_avg

,AVG(diff)
0,-89.125


## Test the hypothesis

In [45]:
query='''
SELECT
    test.uid as uid,
    AVG(CAST((julianday(test.first_commit_ts) - julianday(datetime(deadlines.deadlines, 'unixepoch'))) * 24 AS integer)) AS avg_diff,
    views.av AS pageviews
FROM
    test
JOIN
    deadlines
        ON
            test.labname=deadlines.labs
JOIN (
    SELECT
        uid,
        COUNT(uid) AS av
    FROM
        pageviews
    GROUP BY uid
    )
        AS views
        ON
            test.uid=views.uid
WHERE
    labname != 'project1'
GROUP BY
    test.uid
'''

views_diff = pd.read_sql(sql=query, con=db_connect)
views_diff

,uid,avg_diff,pageviews
0,user_1,-64.400000,28
1,user_10,-74.800000,89
2,user_14,-159.000000,143
3,user_17,-61.600000,47
4,user_18,-5.666667,3
5,user_19,-98.750000,16
6,user_21,-95.500000,10
7,user_25,-92.600000,179
8,user_28,-86.400000,149
9,user_3,-105.400000,317


In [46]:
views_diff.corr()

C:\Users\salav\AppData\Local\Temp\ipykernel_3260\1431885951.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  views_diff.corr()


,avg_diff,pageviews
avg_diff,1.000000,-0.279736
pageviews,-0.279736,1.000000


## Close the connection

In [47]:
db_connect.close()